In [ ]:
#install openai module
!pip install openai

In [71]:
#import modules
from openai import OpenAI
import pandas as pd
client = OpenAI(api_key = 'sk-svcacct-3i9xo16XX9bCG6OxE5LXT3BlbkFJrfdBVEzw9RnMi8pfMtAy')

In [72]:
#load data
reviews = pd.read_csv("book_reviews.csv")
books = pd.read_csv("books.csv")
reviews= reviews.drop(reviews.columns[0], axis = 1)
books= books.drop(books.columns[0], axis = 1)

In [73]:
#%% merge data
books = books[['book title', 'author']]
books.columns=['book name', 'author']
reviews = reviews.merge(books, on = 'book name', how = 'left')

In [87]:
#function to filter required summaries
def filtered_summaries(author_name, titles):
    filtered_df = reviews[reviews['author'] == author_name]
    if titles == 'All':
        return filtered_df
    if isinstance(titles, (list, tuple, set)):
        titles = set(titles)
        filtered_df = filtered_df[filtered_df['book name'].isin(titles)]
        return filtered_df
    titles = str(titles)
    filtered_df = filtered_df[filtered_df['book name'] == titles]
    return filtered_df

#function to colate the filtered summary
def collate_summaries(filtered_df):
    review_summaries_collated = ""
    for i,row in filtered_df.iterrows():
        review_summaries_collated += row['review title'] + " - " + str(row['reviewer rating']) + " - " + row['review description'] + "\n\n"
    return review_summaries_collated


In [75]:
#function to generate summary of review using GPT Model
def generate_response(author_name, collated_summaries):
    try:
        prompt = f"""Below are some collated reviews for a few books of {author_name}.
                    Reviews are in format "Title - Rating - Review"
                    Summarize the top 3-5 positive and top 3-5 negative takeaways from the reviews.
                    Also highlight the improvement areas for the author for his/her next work.
                    Remember to ignore if a comment is irrelavant to the book/work
                    """
        response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": collated_summaries}
                ],
                max_tokens=10000
            )
        return response.choices[0].message.content
    except Exception as e:
        print(e)


In [76]:
#main function to process input
def summarize_books(author_name, titles):
    filtered_df = filtered_summaries(author_name, titles)
    collated_summaries = collate_summaries(filtered_df)
    response = generate_response(author_name, collated_summaries)
    print(response)

In [77]:
#sample input-1
author_name = 'Lucille Colandro'
titles = 'All'

summarize_books(author_name, titles)

### Positive Takeaways:
1. **Engagement and Enjoyment**: Many reviews highlight that children love these books, indicating high engagement and entertainment value, particularly during readings in classrooms or at home.
2. **Educational Value**: The stories are noted for their ability to teach vocabulary and sequencing, making them valuable resources for both teachers and parents.
3. **Illustration Appeal**: The cute pictures and illustrations draw positive feedback, enhancing the overall reading experience for young audiences.
4. **Suitability for Young Readers**: The repetitive phrases and simple language make these books accessible, allowing children to engage in the reading process and even read along.
5. **Positive Classroom Impact**: Teachers find these books suitable for their lessons, particularly around holidays like Thanksgiving, suggesting they fit well into educational curricula.

### Negative Takeaways:
1. **Inconsistent Quality**: Some readers express disappointment with c

In [78]:
#sample input-2
author_name = 'David Grann'
titles = ['Killers of the Flower Moon: The Osage Murders and the Birth of the FBI', 'The Wager: A Tale of Shipwreck, Mutiny and Murder']

summarize_books(author_name, titles)

### Positive Takeaways:
1. **Compelling Narrative**: Many readers found David Grann's works to be gripping and well-researched, effectively blending history and storytelling. His ability to transform historical events into engaging narratives has been praised, making complex topics accessible and relatable.
   
2. **Extensive Research**: Grann's thorough research is a consistent highlight, with reviewers appreciating the depth of detail and the incorporation of primary sources such as personal diaries and documents from the period.

3. **Emotional Impact**: The emotional weight of the subjects he tackles, especially the tragic history of the Osage people, resonates with readers. His writing evokes a strong emotional response and a reflection on historical injustices.

4. **Educational Value**: Reviewers noted that Grann's books offer profound insights into historical events that are often overlooked or forgotten, asserting that his works should be included in educational curricula to r

In [88]:
#sample input-3
author_name = 'David Grann'
titles = 'Killers of the Flower Moon: The Osage Murders and the Birth of the FBI'

summarize_books(author_name, titles)

### Positive Takeaways:
1. **Engaging Writing Style**: Many readers appreciate Grann's smooth, unobtrusive writing style. His ability to make historical events come alive transforms potentially dry subjects into compelling narratives.
   
2. **Deep Research**: Grann’s extensive research into the Osage murders, legal inequalities, and societal corruption is highlighted as a major strength, providing both depth and factual authenticity that enhances the book's overall impact.

3. **Historical Importance**: Reviewers commend the book for shedding light on a dark chapter in American history that is often overlooked. The educational value and relevance of the subject matter amplify its significance for modern readers.

4. **Character Development**: The portrayal of key figures such as Molly Burkhart and Tom White resonated with readers, offering a personal touch to the historical events and making them relatable and profound.

5. **Gripping Narrative**: The book reads like a thriller, with 